# Assignment 2 - Machine Learning
**Project:** Project 1 - Classify Images of Road Traffic Signs

**Group Name:** Dan & Ethan
| Name               | Student Number |
|--------------------|----------------|
| Daniel Schellekens | s3900792       |
| Ethan Rayner       | s3902240       |

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

TEST_TRAIN_SPLIT = 0.5

data = pd.read_csv("images.csv")
train_data, val_data = train_test_split(data, test_size=TEST_TRAIN_SPLIT, random_state=0)
print("Test set: {} rows\nValidation set: {} rows".format(train_data.shape[0], val_data.shape[0]))

Test set: 1850 rows
Validation set: 1850 rows
